importe librerias y tambien un modulo propio que llame funciones

In [5]:
import sys
sys.path.append('..') 
from functions import convert_bin,get_dataframes,get_all_texts_for_dic
import numpy as np

get_dataframes() = es una funcion que esta en el modulo 'funciones'.Tiene tres parametros(ruta,tipo,raiz) y trae un lista de las rutas donde se encuentran los archivos de la base de datos 

In [6]:
df = get_dataframes('arrl-org/arrl-2013','texto','..')
option = df
len(option)

96

In [7]:
type(df)

list

get_texts() = es una funcion que sirve para extraer los textos de cada ruta solisitados y toma de parametros una lista con rutas

In [8]:
def get_texts(list_files_txt):
    frases = []

    print(len(list_files_txt))

    for archivo in list_files_txt:
        with open(archivo, 'r') as file:
            contenido = file.read()
            frases.append(contenido)
            file.close()

    frases_array = []
    
    for frase in frases:
        frase_modificada = frase.replace('\n\n', '').replace('\n', ' ')
        frases_array.append(frase_modificada)
    return frases_array

In [9]:
array_texts = get_texts(option)
array_texts

96


[' ‰  NOW 10 WPM  ‰  TEXT IS FROM JULY 2012 QST  PAGE 55  ‰TELEVISION WAS POPULAR THE TYPICAL TRANSMISSION LINE WAS OPEN WIRE OR TWISTED PAIR CONNECTED TO A LINK COUPLED TUNED RESONANT CIRCUIT AT THE OUTPUT OF THE TRANSMITTER, USUALLY WITH DIFFERENT PLUG IN COILS FOR EACH BAND.  BY ADJUSTING THE VARIABLE CAPACITOR, SOME ADJUSTMENT COULD BE MADE TO COMPENSATE FOR REACTANCE IN THE LOAD.  IN ADDITION, SOME HAD AN ‰  END OF 10 WPM TEXT  ‰  QST DE W1AW  ƒ \x1a',
 ' ‰  NOW 10 WPM  ‰  TEXT IS FROM JULY 2012 QST  PAGE 75  ‰HAD A MILITARY BACKGROUND AND WAS A NATURAL LEADER WHO LED BY EXAMPLE TO PUT OUR RAGTAG TEAM TOGETHER TO GET THE JOB DONE.  I PLAYED HOOKY FROM SCHOOL TO HELP SUPPORT THE SEARCH EFFORT NOT SO MUCH OUT OF A SPIRIT OF PUBLIC SERVICE, BUT TO SEE MY FRIENDS AND PLAY RADIO IN THE FIELD.  EACH OF US WAS ASSIGNED TO THE VARIOUS DISCIPLINES OF EMERGENCY MANAGEMENT, LAW ‰  END OF 10 WPM TEXT  ‰  QST DE W1AW  ƒ \x1a',
 ' ‰  NOW 10 WPM  ‰  TEXT IS FROM JULY 2012 QST  PAGE 55  ‰TELEVISI

clean_text() = esta funcion la hice para sacar las inperfecciones a los textos y toma de parametro una lista con strings. esta funcion no tiene return

In [10]:
def clean_text(array_texts):
    for i, elementos in enumerate(array_texts):
        contador = 0
        contador_2 = 0
        for j, elemento in enumerate(elementos[:100]): 
            if '‰' in elemento:
                contador += elemento.count('‰')
            if '=' in elemento:
                contador_2 += elemento.count('=')
        if contador_2 > 0:
            contador = contador_2 + 10         

        if contador == 3:
            sym_1 = elementos[:100].index('‰')
            sym_2 = elementos[sym_1 + 1:100].index('‰')
            sym_3 = elementos[sym_1 + sym_2 + 2 + 1:100].index('‰')
            array_texts[i] = elementos[sym_1 + sym_2 + sym_3 + 4:]
        elif contador == 2:
            sym_1 = elementos[:100].index('‰')
            sym_2 = elementos[sym_1 + 1:100].index('‰')
            array_texts[i] = elementos[sym_1 + sym_2 + 2:]
        elif contador == 13:
            sym_1 = elementos[:100].index('=')
            sym_2 = elementos[sym_1 + 1:100].index('=')
            sym_3 = elementos[sym_1 + sym_2 + 2 + 1:100].index('=')
            array_texts[i] = elementos[sym_1 + sym_2 + sym_3 + 4:]
        elif contador == 12:
            sym_1 = elementos[:100].index('=')
            sym_2 = elementos[sym_1 + 1:100].index('=')
            array_texts[i] = elementos[sym_1 + sym_2 + 2:]
        else:
            print('error',contador)

    for i in range(len(array_texts)):
        elementos = array_texts[i]
        ultimos_100 = elementos[-100:]
        if '‰' in ultimos_100:
            indice = ultimos_100.index('‰')
            elementos = elementos[:len(elementos) - 100 + indice]
            array_texts[i] = elementos
        elif '=' in ultimos_100:
            indice = ultimos_100.index('=')
            elementos = elementos[:len(elementos) - 100 + indice]
            array_texts[i] = elementos
        else:
            print('not found')

    for i in array_texts:
        i.lstrip()            

In [11]:
clean_text(array_texts)


lista con strings limpios

In [12]:
array_texts

['TELEVISION WAS POPULAR THE TYPICAL TRANSMISSION LINE WAS OPEN WIRE OR TWISTED PAIR CONNECTED TO A LINK COUPLED TUNED RESONANT CIRCUIT AT THE OUTPUT OF THE TRANSMITTER, USUALLY WITH DIFFERENT PLUG IN COILS FOR EACH BAND.  BY ADJUSTING THE VARIABLE CAPACITOR, SOME ADJUSTMENT COULD BE MADE TO COMPENSATE FOR REACTANCE IN THE LOAD.  IN ADDITION, SOME HAD AN ',
 'HAD A MILITARY BACKGROUND AND WAS A NATURAL LEADER WHO LED BY EXAMPLE TO PUT OUR RAGTAG TEAM TOGETHER TO GET THE JOB DONE.  I PLAYED HOOKY FROM SCHOOL TO HELP SUPPORT THE SEARCH EFFORT NOT SO MUCH OUT OF A SPIRIT OF PUBLIC SERVICE, BUT TO SEE MY FRIENDS AND PLAY RADIO IN THE FIELD.  EACH OF US WAS ASSIGNED TO THE VARIOUS DISCIPLINES OF EMERGENCY MANAGEMENT, LAW ',
 'TELEVISION WAS POPULAR THE TYPICAL TRANSMISSION LINE WAS OPEN WIRE OR TWISTED PAIR CONNECTED TO A LINK COUPLED TUNED RESONANT CIRCUIT AT THE OUTPUT OF THE TRANSMITTER, USUALLY WITH DIFFERENT PLUG IN COILS FOR EACH BAND.  BY ADJUSTING THE VARIABLE CAPACITOR, SOME ADJUST

importe algunas funciones nuevas del modulo 'funciones'

In [13]:
from functions import get_dataframes,covert_dic_to_json,get_all_texts_for_dic

este es un bucle con un try para manejar errores donde:
1. consumo la primera ruta
2. busco la lista de todas las rutas de ese año
3. la funcion get_all_text_for_dic tiene integrada get_texts(extrae textos de las rutas) y clean_text(limpia en caso de ser nesecario) y luego las tranforma a diccionario con la keys=arrl-{year} y los values=texto limpio 
4. y por ultimo covert_dic_to_json convierte el diccionario a json 

In [14]:
for year in range(2013, 2023):
    try:
        path_folder = f'arrl-org/arrl-{year}/'
        # path_folder = 'F_10-16x16_Koch_Training'
        # dataframes_audio = get_dataframes(path_folder,'audio')
        dataframes_texto = get_dataframes(path_folder,'texto','..')
        get_dic_textos = get_all_texts_for_dic(dataframes_texto)
        # get_dic_audio = get_all_binarys_for_dic(dataframes_audio)
        covert_dic_to_json(get_dic_textos,f"{path_folder.rstrip('/').split('/')[-1]}-texto")    
    except Exception as e:
        print("error:", str(e))


96
{"130109_10.txt": "TELEVISION WAS POPULAR THE TYPICAL TRANSMISSION LINE WAS OPEN WIRE OR TWISTED PAIR CONNECTED TO A LINK COUPLED TUNED RESONANT CIRCUIT AT THE OUTPUT OF THE TRANSMITTER, USUALLY WITH DIFFERENT PLUG IN COILS FOR EACH BAND.  BY ADJUSTING THE VARIABLE CAPACITOR, SOME ADJUSTMENT COULD BE MADE TO COMPENSATE FOR REACTANCE IN THE LOAD.  IN ADDITION, SOME HAD AN ", "130123_10.txt": "HAD A MILITARY BACKGROUND AND WAS A NATURAL LEADER WHO LED BY EXAMPLE TO PUT OUR RAGTAG TEAM TOGETHER TO GET THE JOB DONE.  I PLAYED HOOKY FROM SCHOOL TO HELP SUPPORT THE SEARCH EFFORT NOT SO MUCH OUT OF A SPIRIT OF PUBLIC SERVICE, BUT TO SEE MY FRIENDS AND PLAY RADIO IN THE FIELD.  EACH OF US WAS ASSIGNED TO THE VARIOUS DISCIPLINES OF EMERGENCY MANAGEMENT, LAW ", "130206_10.txt": "TELEVISION WAS POPULAR THE TYPICAL TRANSMISSION LINE WAS OPEN WIRE OR TWISTED PAIR CONNECTED TO A LINK COUPLED TUNED RESONANT CIRCUIT AT THE OUTPUT OF THE TRANSMITTER, USUALLY WITH DIFFERENT PLUG IN COILS FOR EACH BAN

### vectorizar output(textos)

- esto se hace para que funcione mejor en la red  . A todas las fraces las divide por palabras , esto quiere decir que la matriz del output va a ser de (muestras totales, palabras totales contando todos los ejemplos)


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

matriz_conteo = vectorizer.fit_transform(array_texts)

print(matriz_conteo.toarray().shape)

(96, 2853)


Paso invero a vectorizar

In [18]:
vectorizar = CountVectorizer()

matriz_conteo = vectorizar.fit_transform(array_texts)

strings_decodificados = vectorizar.inverse_transform(matriz_conteo)

print(strings_decodificados)

[array(['television', 'was', 'popular', 'the', 'typical', 'transmission',
       'line', 'open', 'wire', 'or', 'twisted', 'pair', 'connected', 'to',
       'link', 'coupled', 'tuned', 'resonant', 'circuit', 'at', 'output',
       'of', 'transmitter', 'usually', 'with', 'different', 'plug', 'in',
       'coils', 'for', 'each', 'band', 'by', 'adjusting', 'variable',
       'capacitor', 'some', 'adjustment', 'could', 'be', 'made',
       'compensate', 'reactance', 'load', 'addition', 'had', 'an'],
      dtype='<U16'), array(['was', 'the', 'to', 'of', 'in', 'each', 'by', 'had', 'military',
       'background', 'and', 'natural', 'leader', 'who', 'led', 'example',
       'put', 'our', 'ragtag', 'team', 'together', 'get', 'job', 'done',
       'played', 'hooky', 'from', 'school', 'help', 'support', 'search',
       'effort', 'not', 'so', 'much', 'out', 'spirit', 'public',
       'service', 'but', 'see', 'my', 'friends', 'play', 'radio', 'field',
       'us', 'assigned', 'various', 'discipline